<a href="https://colab.research.google.com/github/NI8868/AnalyticsInPractice2425/blob/main/SQLite_DB_Exercise_GH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**STEP 1: CREATE the SQLite database;**


We need to import the sqlite3 module and create the database and tables.  You'll see this follows the syntax we have used on previous weeks.

Note that we have created the student table with a primary key that is not an INTEGER.

Is this good practice?  
What are the issues and benefits of doing this?

In [1]:
import sqlite3

#This statement creates a connection labelled as conn.  This will be used throughout to ensure the consistency for when we start to query the database tables.
conn = sqlite3.connect('student_grades.db')
cursor = conn.cursor()

#create the student table - we've set the ID to be a primary key.  Is it good to create the primary key as an TEXT string.
cursor.execute('''
CREATE TABLE IF NOT EXISTS student (
  ID TEXT PRIMARY KEY,
  First TEXT NOT NULL,
  Last TEXT NOT NULL
)
''')

#create the grade table - no primary key provided.  As students can exist multiple times in the table as can a course.
cursor.execute('''
CREATE TABLE IF NOT EXISTS grade (
  ID TEXT,
  Code TEXT NOT NULL,
  Mark INTEGER NOT NULL
)
''')

#create the course table - primary key provided again set as TEXT.
cursor.execute('''
CREATE TABLE IF NOT EXISTS course (
  Code TEXT PRIMARY KEY,
  Title TEXT NOT NULL
)
''')

#This saves the chnages to the databae.  Up unitl this point the executed SQL statement isn't stored, changes are not immediatley saved.
conn.commit()
#conn.close()
print("Database and tables created successfully!")


Database and tables created successfully!


**STEP 2: Check Tables Created:**

Run the command to show the database tables created and the structure.

In [2]:
# prompt: show the table structures

#import sqlite3

#conn = sqlite3.connect('student_grades.db')
#cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

for table_name in tables:
    print(f"Table: {table_name[0]}")
    cursor.execute(f"PRAGMA table_info({table_name[0]});")
    columns = cursor.fetchall()
    for col in columns:
        print(f"  Column: {col[1]}, Type: {col[2]}, NotNull: {col[3]}, DefaultVal: {col[4]}, PrimaryKey: {col[5]}")
    print("-" * 20)

#conn.close()


Table: student
  Column: ID, Type: TEXT, NotNull: 0, DefaultVal: None, PrimaryKey: 1
  Column: First, Type: TEXT, NotNull: 1, DefaultVal: None, PrimaryKey: 0
  Column: Last, Type: TEXT, NotNull: 1, DefaultVal: None, PrimaryKey: 0
--------------------
Table: grade
  Column: ID, Type: TEXT, NotNull: 0, DefaultVal: None, PrimaryKey: 0
  Column: Code, Type: TEXT, NotNull: 1, DefaultVal: None, PrimaryKey: 0
  Column: Mark, Type: INTEGER, NotNull: 1, DefaultVal: None, PrimaryKey: 0
--------------------
Table: course
  Column: Code, Type: TEXT, NotNull: 0, DefaultVal: None, PrimaryKey: 1
  Column: Title, Type: TEXT, NotNull: 1, DefaultVal: None, PrimaryKey: 0
--------------------


**STEP 3: Upload Files:**

Run this box three times to upload the relevant csv files.

Course_Table.csv, Student_Table.csv & Grade_table.csv

In [3]:


from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))


Saving Course_Table.csv to Course_Table (1).csv
Saving Grade_Table.csv to Grade_Table (1).csv
Saving Student_Table.csv to Student_Table (1).csv
User uploaded file "Course_Table (1).csv" with length 614 bytes
User uploaded file "Grade_Table (1).csv" with length 12019 bytes
User uploaded file "Student_Table (1).csv" with length 816 bytes


**STEP 4: Load CSV files into the database tables:**

This will populate the database tables with the data from the csv files.  No need to write INSERT statements.

You need to make sure the correct files are loaded into the corresponding tables.

In [4]:
import csv
def import_csv_to_table(csv_file, table_name):
    #opens the file as read only 'r', doesn't allow the origianl csv to be changed.
    with open(csv_file, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        next(csv_reader)  # Skip header row if present
        for row in csv_reader:
            #? creates a placeholder for each column in the CSV file. ['?','?','?'] - Join makes it a string so it can then be inserted.
            # use of the '?' reduce risk of SQL injection
            placeholders = ', '.join(['?' for _ in row])
            #Assumes that the CSV and table have the same structure (this could be an issue) Would have to specify column names if different.
            sql = f"INSERT INTO {table_name} VALUES ({placeholders})"
            cursor.execute(sql, row)

# Import data from CSV files into the relevant table - Student_Table goes into student table.  Teh import_csv_to_table is the function, passing the two values across.
try:
    import_csv_to_table('Student_Table.csv', 'student')
    import_csv_to_table('Course_Table.csv', 'course')
    import_csv_to_table('Grade_Table.csv', 'grade')
    conn.commit()
    print("Data imported successfully!")
except Exception as e:
    print(f"An error occurred: {e}")
    conn.rollback()  # Rollback changes if an error occurred



Data imported successfully!


**STEP 5: Check Data has loaded**

Query each database table and load the data into a dataframe and display the first 5 lines

In [5]:
import pandas as pd
# Query all three tables and load into pandas DataFrames
student_df = pd.read_sql_query("SELECT * FROM student", conn)
grade_df = pd.read_sql_query("SELECT * FROM grade", conn)
course_df = pd.read_sql_query("SELECT * FROM course", conn)

# Show the first 5 lines of each DataFrame
print("Student Table:")
print(student_df.head(5))
print("\nGrade Table:")
print(grade_df.head(5))
print("\nCourse Table:")
print(course_df.head(5))




Student Table:
      ID    First     Last
0  S1001    Alice    Smith
1  S1002      Bob    Brown
2  S1003  Charlie  Johnson
3  S1004    David   Taylor
4  S1005     Emma    Brown

Grade Table:
      ID  Code  Mark
0  S1005  C123    55
1  S1005  C124    76
2  S1011  C124    47
3  S1014  C117    50
4  S1032  C116    54

Course Table:
   Code                Title
0  C101      Mathematics 101
1  C102  Physics Inroduction
2  C103  Chemistry Practical
3  C104     Computer Science
4  C105  Economics & Finance


**ONLY RUN IF YOU NEED TO DELETE THE DATA IN THE TABLES**

If you run go back to **STEP 4** and re-run from there.

In [ ]:
# only run if you need to reset the tables without deleting the database and starting again - then re-run the box previous box.
# Delete all data from the tables
cursor.execute("DELETE FROM student")
cursor.execute("DELETE FROM grade")
cursor.execute("DELETE FROM course")

conn.commit()
print("All data deleted from the tables successfully!")



All data deleted from the tables successfully!


**STEP 6: SQL Select statements**

Run the following statements.  Please ask yoursefl the impact of each one before running.




In [7]:
# Select all columns from the student table
student_df = pd.read_sql_query("SELECT * FROM student", conn)

#What should the output be.
print(student_df)



       ID    First       Last
0   S1001    Alice      Smith
1   S1002      Bob      Brown
2   S1003  Charlie    Johnson
3   S1004    David     Taylor
4   S1005     Emma      Brown
5   S1006    Fiona     Thomas
6   S1007   George      White
7   S1008   Hannah     Harris
8   S1009    Isaac      Smith
9   S1010    Julia   Thompson
10  S1011      Bob      Lewis
11  S1012    Laura      Smith
12  S1013  Michael      Allen
13  S1014    Nancy      Young
14  S1015   Oliver       King
15  S1016    Paula      Smith
16  S1017   Robert      Scott
17  S1018   Robert      Adams
18  S1019   Sophia      Baker
19  S1020  Timothy      Brown
20  S1021  Timothy       Hall
21  S1022   Victor     Carter
22  S1023    Wendy     Foster
23  S1024  Yasmine      Price
24  S1025  Yasmine    Roberts
25  S1026  Zachary      Brown
26  S1027    Aaron      Evans
27  S1028     Beth   Campbell
28  S1029  Yasmine     Parker
29  S1030    Diana     Murphy
30  S1031    Ethan      Price
31  S1032  Felicia     Foster
32  S1033 

In [8]:
# Select Last from the student table
studentLast_df = pd.read_sql_query("SELECT Last FROM student", conn)

#What should the output be.
print(studentLast_df)



         Last
0       Smith
1       Brown
2     Johnson
3      Taylor
4       Brown
5      Thomas
6       White
7      Harris
8       Smith
9    Thompson
10      Lewis
11      Smith
12      Allen
13      Young
14       King
15      Smith
16      Scott
17      Adams
18      Baker
19      Brown
20       Hall
21     Carter
22     Foster
23      Price
24    Roberts
25      Brown
26      Evans
27   Campbell
28     Parker
29     Murphy
30      Price
31     Foster
32     Bryant
33  Alexander
34    Russell
35     Foster
36     Foster
37      Hayes
38   Sullivan
39      Price


In [10]:
# Select DISTINCT last names from the student table
studentLastUnique_df = pd.read_sql_query("SELECT DISTINCT First FROM student", conn)

#What should the output be.   What does this tell you from the previous outputs?
print(studentLastUnique_df)



      First
0     Alice
1       Bob
2   Charlie
3     David
4      Emma
5     Fiona
6    George
7    Hannah
8     Isaac
9     Julia
10    Laura
11  Michael
12    Nancy
13   Oliver
14    Paula
15   Robert
16   Sophia
17  Timothy
18   Victor
19    Wendy
20  Yasmine
21  Zachary
22    Aaron
23     Beth
24    Diana
25    Ethan
26  Felicia
27   Gordon
28      Ian
29  Jasmine
30     Kyle
31     Matt
32     Nina


In [14]:
# Select DISTINCT First from the student table - modify the query
studentFirstUnique_df = pd.read_sql_query("SELECT DISTINCT First FROM student", conn)

#What should the output be?  What does this tell you from the previous outputs?
print(studentFirstUnique_df)

      First
0     Alice
1       Bob
2   Charlie
3     David
4      Emma
5     Fiona
6    George
7    Hannah
8     Isaac
9     Julia
10    Laura
11  Michael
12    Nancy
13   Oliver
14    Paula
15   Robert
16   Sophia
17  Timothy
18   Victor
19    Wendy
20  Yasmine
21  Zachary
22    Aaron
23     Beth
24    Diana
25    Ethan
26  Felicia
27   Gordon
28      Ian
29  Jasmine
30     Kyle
31     Matt
32     Nina


**STEP 7: SELECT with WHERE**

In [15]:
# Select Marks that are over 60.
studentWhere_df = pd.read_sql_query("SELECT Mark FROM grade WHERE (Mark >= 60)", conn)

#What should the output be.
print(studentWhere_df)

     Mark
0      76
1      75
2      72
3      61
4      72
..    ...
420    75
421    71
422    60
423    67
424    74

[425 rows x 1 columns]


In [17]:
#export the dataframe to csv for further analysis
#the index=False means no row numbers are exported.
#change index to True and compare the outputs.  Youll need to download formthe Files window

studentWhere_df.to_csv('Grades_Over_60.csv', index=True)


**TASK**

Create a statement to select all students that have passed the Finance Management Course and export the file to CSV.

Just using standard select statments.

How would you tackle the problem.

In [18]:
#review the structure of the grades table and the course table
print("\nGrade Table:")
print(grade_df.head(5))
print("\nCourse Table:")
print(course_df.head(5))


Grade Table:
      ID  Code  Mark
0  S1005  C123    55
1  S1005  C124    76
2  S1011  C124    47
3  S1014  C117    50
4  S1032  C116    54

Course Table:
   Code                Title
0  C101      Mathematics 101
1  C102  Physics Inroduction
2  C103  Chemistry Practical
3  C104     Computer Science
4  C105  Economics & Finance


In [20]:
#select the course code from the course table for the Finance Management Course
studentCourse_df = pd.read_sql_query("SELECT * FROM Course WHERE Title = 'Finance Management'", conn)

print(studentCourse_df)

   Code               Title
0  C115  Finance Management


In [24]:
#select student ID and Mark for the Finance Management Course
studentCourseMark_df = pd.read_sql_query("SELECT ID,Mark FROM Grade WHERE Code = 'C115' AND Mark >= '50'", conn)

print(studentCourseMark_df)

       ID  Mark
0   S1034    57
1   S1012    68
2   S1017    51
3   S1031    60
4   S1036    58
5   S1028    61
6   S1005    67
7   S1009    59
8   S1008    59
9   S1038    75
10  S1023    57
11  S1040    65
12  S1019    58
13  S1026    53
14  S1013    63
15  S1033    72
16  S1002    57
17  S1006    62
18  S1022    74
19  S1014    72
20  S1016    54
21  S1018    77
22  S1020    64
23  S1029    57
24  S1011    54
25  S1003    68
26  S1004    75


**STEP 8: Select from multiple tables in one statement**

This can cause issues where tables have the same column names in different tables.  

To resolve this we need to make use of the following syntax:
TableName.Column

In [25]:
#select the following coloumns:  First, Last & Mark from the Student and Grade tables
studentMarks_df = pd.read_sql_query("SELECT First, Last, Mark FROM student, grade WHERE (student.ID = grade.ID)", conn)

print(studentMarks_df)


       First      Last  Mark
0       Emma     Brown    55
1       Emma     Brown    76
2        Bob     Lewis    47
3      Nancy     Young    50
4    Felicia    Foster    54
..       ...       ...   ...
795     Kyle    Foster    54
796     Matt  Sullivan    57
797    Laura     Smith    74
798    Diana    Murphy    53
799     Kyle    Foster    54

[800 rows x 3 columns]


**TASK**

Modify the statement to get all the students that acheived a mark over 50

In [26]:
#select the following coloumns:  First, Last & Mark from the Student and Grade tables
studentMarks_df = pd.read_sql_query("SELECT First, Last, Mark FROM student, grade WHERE (Student.ID = Grade.ID) AND Mark>='50'", conn)

print(studentMarks_df)

       First       Last  Mark
0       Emma      Brown    55
1       Emma      Brown    76
2      Nancy      Young    50
3    Felicia     Foster    54
4       Beth  Alexander    75
..       ...        ...   ...
633     Kyle     Foster    54
634     Matt   Sullivan    57
635    Laura      Smith    74
636    Diana     Murphy    53
637     Kyle     Foster    54

[638 rows x 3 columns]


In [30]:
#select the following coloumns:  First, Last & Mark from the Student and Grade tables
studentMarks_df = pd.read_sql_query("SELECT First, Last, Mark FROM student, grade WHERE (Student.ID = Grade.ID) AND Mark>='50'", conn)

print(studentMarks_df)

       First       Last  Mark
0       Emma      Brown    55
1       Emma      Brown    76
2      Nancy      Young    50
3    Felicia     Foster    54
4       Beth  Alexander    75
..       ...        ...   ...
633     Kyle     Foster    54
634     Matt   Sullivan    57
635    Laura      Smith    74
636    Diana     Murphy    53
637     Kyle     Foster    54

[638 rows x 3 columns]


In [31]:
#modify the query to gather the 'Statistics For Python' Course and all studnet that obtained a mark between 55 and 65
studentMarks_df = pd.read_sql_query("SELECT First, Last, Mark FROM student, grade, course WHERE (Student.ID = Grade.ID) AND (course.Title ='Statistics For Python') AND (Mark BETWEEN 55 AND 65)", conn)

print(studentMarks_df)

#AND是把所有符合条件的都选一起

       First       Last  Mark
0       Emma      Brown    55
1       Beth   Campbell    61
2       Beth  Alexander    57
3    Charlie    Johnson    55
4     George      White    59
..       ...        ...   ...
252     Beth   Campbell    63
253     Matt   Sullivan    65
254  Timothy       Hall    60
255   Victor     Carter    56
256     Matt   Sullivan    57

[257 rows x 3 columns]


**STEP 9: ORDER BY statements**

Can be set to be either ASC or DESC.  The syntax is ORDER BY added to the select statement.  The default setting if not included is ASC.

In [32]:
studentMarks_df = pd.read_sql_query("SELECT * FROM grade ORDER BY Mark", conn)

print (studentMarks_df)

        ID  Code  Mark
0    S1027  C107    35
1    S1014  C103    35
2    S1032  C118    35
3    S1036  C108    35
4    S1018  C114    37
..     ...   ...   ...
795  S1019  C113    81
796  S1020  C113    81
797  S1023  C109    82
798  S1029  C105    82
799  S1001  C117    82

[800 rows x 3 columns]


**TASK**

Modify the statement to order by Course Code and then Mark

In [36]:
studentMarks_df = pd.read_sql_query("SELECT * FROM grade ORDER BY Code, Code DESC, Mark ASC", conn)
print (studentMarks_df)
#Code DESC, Mark DESC分别说

        ID  Code  Mark
0    S1023  C101    45
1    S1032  C101    45
2    S1024  C101    45
3    S1036  C101    45
4    S1037  C101    46
..     ...   ...   ...
795  S1014  C124    73
796  S1038  C124    74
797  S1005  C124    76
798  S1009  C124    80
799  S1037  C124    81

[800 rows x 3 columns]


Modify the statement to order by Code, Mark and obtain the student Name.  Only show Name and Mark for Course Id

In [38]:
studentMarks_df = pd.read_sql_query("SELECT Last, First, Mark FROM student, grade WHERE (student.ID = grade.ID) ORDER BY Code DESC, Mark DESC", conn)
print (studentMarks_df)

       Last    First  Mark
0    Foster     Kyle    81
1     Smith    Isaac    80
2     Brown     Emma    76
3     Hayes     Beth    74
4     Young    Nancy    73
..      ...      ...   ...
795  Taylor    David    46
796  Foster    Wendy    45
797  Foster  Felicia    45
798   Price  Yasmine    45
799  Foster  Jasmine    45

[800 rows x 3 columns]


Modify the statement to select for a specific course and display the course code and title, order by the grade ASC - Statistics For Python

In [44]:
studentMarks_df = pd.read_sql_query("SELECT grade.Code, course.Title, student.Last, student.First, grade.Mark FROM student INNER JOIN grade ON student.ID = grade.ID INNER JOIN course ON grade.Code = course.Code WHERE course.Title = 'Statistics For Python' ORDER BY grade.Mark ASC", conn)
print (studentMarks_df)

    Code                  Title       Last    First  Mark
0   C113  Statistics For Python    Roberts  Yasmine    37
1   C113  Statistics For Python   Thompson    Julia    37
2   C113  Statistics For Python  Alexander     Beth    39
3   C113  Statistics For Python     Carter   Victor    41
4   C113  Statistics For Python      Price     Nina    45
5   C113  Statistics For Python      Lewis      Bob    46
6   C113  Statistics For Python     Foster     Kyle    49
7   C113  Statistics For Python      Smith    Paula    52
8   C113  Statistics For Python      Price  Yasmine    54
9   C113  Statistics For Python      Scott   Robert    54
10  C113  Statistics For Python      Price    Ethan    55
11  C113  Statistics For Python      Allen  Michael    57
12  C113  Statistics For Python   Sullivan     Matt    60
13  C113  Statistics For Python   Campbell     Beth    64
14  C113  Statistics For Python    Johnson  Charlie    64
15  C113  Statistics For Python      Young    Nancy    64
16  C113  Stat

In [40]:
studentMarks_df = pd.read_sql_query("SELECT , First, Mark FROM student, grade，course WHERE (student.ID = grade.ID) ORDER BY Mark ASC", conn)
print (studentMarks_df)

       Last    First  Mark
0     Evans    Aaron    35
1     Young    Nancy    35
2    Foster  Felicia    35
3    Foster  Jasmine    35
4     Adams   Robert    37
..      ...      ...   ...
795   Baker   Sophia    81
796   Brown  Timothy    81
797  Foster    Wendy    82
798  Parker  Yasmine    82
799   Smith    Alice    82

[800 rows x 3 columns]


**STEP 10: Arthimatic and Aggregating functions**

Simple arithmatic on the columns.  

When running we make use of AS - to make better readibility for the column name.

In [ ]:
#simple Adding values, Subtracting Values, Multiplyig Values:
studentCount_df = pd.read_sql_query("SELECT Mark, Mark/2 AS DIVIDED, Mark*2 AS DOUBLED, Mark+10 AS MODERATED FROM grade", conn)
print (studentCount_df)

In [ ]:
#simple Count:  Count up the rows:
studentCount_df = pd.read_sql_query("SELECT COUNT(*) AS COUNT FROM grade", conn)
print (studentCount_df)

In [ ]:
#simple SUM for the column values
studentCount_df = pd.read_sql_query("SELECT SUM(mark) AS TOTAL FROM grade", conn)
print (studentCount_df)

In [ ]:
#simple MAX for the column values
studentCount_df = pd.read_sql_query("SELECT MAX(mark) AS BEST FROM grade", conn)
print (studentCount_df)

#modify to print the Lowest Mark

**TASK**

How would you calculate the RANGE of values in the marks column.

In [45]:
studentCount_df = pd.read_sql_query("SELECT MAX(Mark)-MIN(Mark) AS RANGE FROM grade", conn)
print (studentCount_df)

   RANGE
0     47


How would you find a specific students marks for all modules taken and the average mark.

The student is Laura Smith



In [51]:
studentMarks_df = pd.read_sql_query("SELECT First AS FROM grade, student, course Where (student.id= grade.id) AND (course.code= grade.code) AND (course.title = 'Statisitcs For Python') ORDER BY Mark ASC", conn)
print (studentMarks_df)

DatabaseError: Execution failed on sql 'SELECT First AS FROM grade, student, course Where (student.id= grade.id) AND (course.code= grade.code) AND (course.title = 'Statisitcs For Python') ORDER BY Mark ASC': near "FROM": syntax error

**STEP 11: Group BY Statements**

When we need to pull a group of rows together and carry out an aggregation of data.

In [49]:
studentGroup_df = pd.read_sql_query("SELECT First, Last, AVG(Mark), Code AS AVERAGE FROM student,grade GROUP BY Code", conn)
print (studentGroup_df)

    First   Last  AVG(Mark) AVERAGE
0   Alice  Smith  59.823529    C101
1   Alice  Smith  62.366667    C102
2   Alice  Smith  57.911765    C103
3   Alice  Smith  61.228571    C104
4   Alice  Smith  60.515152    C105
5   Alice  Smith  57.909091    C106
6   Alice  Smith  57.900000    C107
7   Alice  Smith  58.540541    C108
8   Alice  Smith  59.638889    C109
9   Alice  Smith  59.906250    C110
10  Alice  Smith  61.424242    C111
11  Alice  Smith  60.028571    C112
12  Alice  Smith  60.781250    C113
13  Alice  Smith  64.093750    C114
14  Alice  Smith  60.218750    C115
15  Alice  Smith  58.542857    C116
16  Alice  Smith  61.393939    C117
17  Alice  Smith  60.542857    C118
18  Alice  Smith  62.000000    C119
19  Alice  Smith  60.638889    C120
20  Alice  Smith  58.361111    C121
21  Alice  Smith  63.709677    C122
22  Alice  Smith  61.600000    C123
23  Alice  Smith  60.093750    C124


**TASK**

Create a query to show the First, Last name of the student and their average grade across all modules and how many moudles have been taken.

In [53]:
studentGroup_df = pd.read_sql_query("SELECT First, Last,AVG(Mark) AS AVERAGE FROM student, grade WHERE (student.id= grade.id) GROUP BY Last, First ORDER BY AVERAGE DESC", conn)
print (studentGroup_df)

#modify the query to order by AVERAGE grade DESC

      First       Last  AVERAGE
0   Yasmine     Parker    65.35
1    Robert      Adams    64.20
2     Isaac      Smith    63.95
3      Beth      Hayes    63.55
4     David     Taylor    63.50
5      Beth  Alexander    63.15
6    Sophia      Baker    62.90
7   Michael      Allen    62.70
8   Timothy       Hall    62.55
9     Wendy     Foster    62.40
10    Nancy      Young    62.30
11    Aaron      Evans    62.05
12    Laura      Smith    61.90
13   Oliver       King    61.70
14      Bob      Brown    61.60
15    Ethan      Price    61.55
16  Yasmine      Price    61.05
17   Victor     Carter    61.00
18  Felicia     Foster    60.95
19  Charlie    Johnson    60.85
20  Timothy      Brown    60.70
21     Emma      Brown    60.65
22    Diana     Murphy    60.25
23    Fiona     Thomas    60.00
24     Beth   Campbell    59.65
25     Kyle     Foster    59.40
26    Alice      Smith    58.90
27    Paula      Smith    58.70
28   Hannah     Harris    58.60
29   Gordon     Bryant    58.35
30   Geo

We can use HAVING to replace a WHERE when we have items that have been grouped together.

In [55]:
#using your above query identify stuent that have an AVG of more then 65
studentGroup_df = pd.read_sql_query("SELECT First, Last,AVG(Mark) AS AVERAGE FROM student, grade WHERE (student.id= grade.id) GROUP BY student.ID HAVING (AVG(Mark)>=65) ORDER BY AVERAGE DESC", conn)
print (studentGroup_df)

#how would you change this so it's between 52 AND 58

     First    Last  AVERAGE
0  Yasmine  Parker    65.35


**STEP 12: Using JOINS**

Cross JOIN A & B - returns all pairs of rows from A and B

Natural JOIN A & B - returns pairs of rows with common values for identical names columns and without dupilcating columns

Inner JOIN A & B - returns pairs of rows satisfying a condition

In [ ]:
#CROSS JOIN
studentJoin_df = pd.read_sql_query("SELECT * FROM student CROSS JOIN grade", conn)
print (studentJoin_df)

In [ ]:
#NATURAL JOIN
studentJoin_df = pd.read_sql_query("SELECT * FROM student NATURAL JOIN grade", conn)
print (studentJoin_df)

In [ ]:
#INNER JOIN
studentJoin_df = pd.read_sql_query("SELECT * FROM student INNER JOIN grade USING (ID)", conn)
print (studentJoin_df)

**STEP 13: Using a WHEN Statement**

The WHEN statement provides the ablity to develop categorising of data, conditional calculations, and filtering results.

It works similar in function to an IF .. ELSE statement.

Data Classification:

In [56]:
# SQL query with CASE statement to classify marks
query = """
SELECT
    student.First,
    student.Last,
    grade.Mark,
    CASE
        WHEN grade.Mark < 50 THEN 'FAIL'
        WHEN grade.Mark BETWEEN 50 AND 59 THEN 'PASS'
        WHEN grade.Mark BETWEEN 60 AND 69 THEN 'PASS'
        WHEN grade.Mark BETWEEN 70 AND 79 THEN 'DISTINCTION'
        ELSE 'NOT ACHIEVABLE'
    END AS Grade
FROM student
JOIN grade ON student.ID = grade.ID
WHERE grade.code = 'C115';
"""

# Execute query
studentMark_df = pd.read_sql_query(query, conn)
print(studentMark_df)

      First       Last  Mark        Grade
0      Beth  Alexander    57         PASS
1     Laura      Smith    68         PASS
2    Robert      Scott    51         PASS
3     Ethan      Price    60         PASS
4   Jasmine     Foster    58         PASS
5     Diana     Murphy    48         FAIL
6      Beth   Campbell    61         PASS
7      Emma      Brown    67         PASS
8    Oliver       King    46         FAIL
9     Isaac      Smith    59         PASS
10    Alice      Smith    46         FAIL
11   Hannah     Harris    59         PASS
12     Beth      Hayes    75  DISTINCTION
13    Wendy     Foster    57         PASS
14     Nina      Price    65         PASS
15   Sophia      Baker    58         PASS
16  Zachary      Brown    53         PASS
17  Michael      Allen    63         PASS
18   Gordon     Bryant    72  DISTINCTION
19      Bob      Brown    57         PASS
20    Fiona     Thomas    62         PASS
21   Victor     Carter    74  DISTINCTION
22    Nancy      Young    72  DIST

Conditional Calculations:

In [ ]:
# SQL query with CASE statement to classify marks
query = """SELECT
    Code,
    Mark,
    ID,
    CASE
        WHEN Mark >= 70 THEN Mark * 0.90  -- 10% reduction
        WHEN Mark <= 50 THEN Mark * 0.95   -- 5% reduction
        ELSE Mark  -- No change
    END AS Moderated_Mark
FROM grade;
"""

# Execute query
studentMark_df = pd.read_sql_query(query, conn)
print(studentMark_df)

Running Multiple Quieries:

In [ ]:
query = """
SELECT
    ID,
    Code,
    Mark,
    CASE Code
        WHEN 'C123' THEN 'Finance'
        WHEN 'C104' THEN 'Course 120'
        WHEN 'C117' THEN 'Course 103'
        ELSE 'Unknown Status'
    END AS UpdatedStatus
FROM grade;
"""

# Execute query
studentMark_df = pd.read_sql_query(query, conn)
print(studentMark_df)